# Coral Reef Health Monitoring around Vanuatu Coastline

In [ ]:
!mamba install --channel rapidsai --quiet --yes cuml

In [ ]:
!mamba install --channel conda-forge libgdal openjpeg gdal rasterio --yes

In [ ]:
!mamba list libgdal-jp2openjpeg


In [ ]:
!ls /srv/conda/envs/notebook/lib/gdalplugins


In [ ]:
import os
os.environ["GDAL_DRIVER_PATH"] = "/srv/conda/envs/notebook/lib/gdalplugins"

from osgeo import gdal
gdal.AllRegister()

drivers = [gdal.GetDriver(i).GetDescription() for i in range(gdal.GetDriverCount())]
print("JP2OpenJPEG" in drivers)
print("JP2KAK" in drivers)


In [ ]:
import joblib
import geopandas as gpd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from odc.stac import load
from pystac_client import Client
from shapely.geometry import mapping, shape, MultiPolygon, Polygon, box
from rasterio.features import geometry_mask
import rasterio
from geocube.api.core import make_geocube
from dask_ml.model_selection import train_test_split
from cuml import RandomForestClassifier
import cupy as cp
import rioxarray 
import xarray as xr
import hvplot.xarray
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    accuracy_score,
    classification_report,
)
from tqdm import tqdm  # for progress bar

#### Define AOI — Vanuatu Coastal Buffer

In [ ]:
# Load province boundaries of Vanuatu
provinces = gpd.read_file("./2016_phc_vut_pid_4326.geojson")

In [ ]:
provinces

In [ ]:
provinces.iloc[-1].geometry

Buffer one province for demonstration.

In [ ]:
PROVINCE = "TAFEA"
DATERANGE_START = "2020-07-01"
DATERANGE_END = "2020-08-30"

In [ ]:
# Reproject to meters
province_proj = provinces.to_crs(epsg=32759) # UTM for Vanuatu

# Get the boundary (coastline)
coastline_proj = province_proj.boundary

# Buffer outward
full_buffer = coastline_proj.buffer(4000)  # 1000 meters

# Subtract the original geometry to get *only the outer shell*
external_only = full_buffer.difference(province_proj.geometry)

# Back to lat/lon
external_only_latlon = external_only.to_crs(epsg=4326)

# Filter to one province (e.g., TORBA)
province_match = provinces[provinces["pname"] == PROVINCE]
external_match = external_only_latlon.loc[province_match.index].iloc[0]

# Ensure MultiPolygon, then get smallest
geom = external_match
#if isinstance(geom, MultiPolygon):
#    selected = min(geom.geoms, key=lambda p: p.area)
#else:
#    selected = geom

if isinstance(geom, MultiPolygon):
    # Sort polygons by area
    sorted_polys = sorted(geom.geoms, key=lambda p: p.area)
    n = len(sorted_polys)
    median_index = n // 2  # Integer division

    # If even number of polygons, pick the lower-middle one
    selected = sorted_polys[median_index - 1] if n % 2 == 0 else sorted_polys[median_index]
else:
    selected = geom  # If it's just a single Polygon

In [ ]:
selected

#### Search STAC for Sentinel-2 L1C Imagery

In [ ]:
# Connect to STAC API
stac = Client.open("https://earth-search.aws.element84.com/v1") 
# https://earth-search.aws.element84.com/v1/collections/sentinel-2-l1c

# Search for Sentinel-2 L1C within the reef buffer
items_l1c = stac.search(
    collections=["sentinel-2-l1c"],
    intersects=selected,
    datetime=f"{DATERANGE_START}/{DATERANGE_END}",
    query={"eo:cloud_cover": {"lt": 20}}
).item_collection()

# Search for Sentinel-2 L2A within the reef buffer to obtain the LCL band for cloud masking
items_l2a = stac.search(
    collections=["sentinel-2-l2a"],
    intersects=selected,
    datetime=f"{DATERANGE_START}/{DATERANGE_END}",
    query={"eo:cloud_cover": {"lt": 20}}
).item_collection()

In [ ]:
len(items_l1c), len(items_l2a)

In [ ]:
items_l1c[0]

In [ ]:
items_l2a[0]

#### Load Imagery

In [ ]:
ds_l1c = load(
    items_l1c,
    bands=["blue", "green", "red", "nir", "swir16"],
    crs="EPSG:32759",  # UTM zone for Vanuatu
    resolution=10,
    #chunks={'x': 1024, 'y': 1024, 'bands': -1, 'time': -1}, #{"time": 1},
    bbox=selected.bounds #aoi.total_bounds  # constrain to buffered coastline
)

ds_l2a = load(
    items_l2a,
    bands=["blue", "green", "red",],
    crs="EPSG:32759",  # UTM zone for Vanuatu
    resolution=10,
    #chunks={'x': 1024, 'y': 1024, 'bands': -1, 'time': -1}, #{"time": 1},
    bbox=selected.bounds #aoi.total_bounds  # constrain to buffered coastline
)

ds_scl = load(
    items_l2a,
    bands=["scl"],
    crs="EPSG:32759",  # UTM zone for Vanuatu
    resolution=10,
    #chunks={'x': 1024, 'y': 1024, 'bands': -1, 'time': -1}, #{"time": 1},
    bbox=selected.bounds #aoi.total_bounds  # constrain to buffered coastline
)

In [ ]:
# Reproject the coastal buffer to match the dataset CRS
buffer_gdf = gpd.GeoDataFrame(geometry=[selected], crs="EPSG:4326")
buffer_proj = buffer_gdf.to_crs(ds_l1c.rio.crs)

In [ ]:
# Create binary mask: True = outside buffer, False = inside
mask_l1c = geometry_mask(
    geometries=[mapping(buffer_proj.iloc[0].geometry)],
    transform=ds_l1c.odc.transform,
    out_shape=(ds_l1c.sizes["y"], ds_l1c.sizes["x"]),
    invert=True  # We want True = inside the buffer
)

mask_l2a = geometry_mask(
    geometries=[mapping(buffer_proj.iloc[0].geometry)],
    transform=ds_l2a.odc.transform,
    out_shape=(ds_l2a.sizes["y"], ds_l2a.sizes["x"]),
    invert=True  # We want True = inside the buffer
)

In [ ]:
# Convert to DataArray
mask_xr_l1c = xr.DataArray(
    mask_l1c,
    dims=("y", "x"),
    coords={"y": ds_l1c.y, "x": ds_l1c.x}
)

mask_xr_l2a = xr.DataArray(
    mask_l2a,
    dims=("y", "x"),
    coords={"y": ds_l2a.y, "x": ds_l2a.x}
)

In [ ]:
ds_masked_l1c = ds_l1c.where(mask_xr_l1c)
ds_masked_l2a = ds_l2a.where(mask_xr_l2a)

In [ ]:
# Define cloud class codes
cloud_classes = [8, 9, 10]

# Create a mask: True = valid (non-cloud), False = cloud
valid_mask = ~ds_scl["scl"].isin(cloud_classes)

In [ ]:
ds_masked = ds_masked_l1c[["blue", "green", "red", "nir", "swir16"]].where(valid_mask)

In [ ]:
ds_masked

In [ ]:
# Choose time slice and scale reflectance
ds_rgb = ds_masked_l2a[["red", "green", "blue"]].isel(time=0)
scale = ds_rgb.to_array().quantile(0.99).item() #.compute().item()  # auto-scale to 99th percentile
ds_rgb = (ds_rgb / scale).clip(0, 1)

# Stack and plot
rgb = xr.concat([ds_rgb["red"], ds_rgb["green"], ds_rgb["blue"]], dim="band")
rgb = rgb.assign_coords(band=["red", "green", "blue"])

rgb.hvplot.rgb(x="x", y="y", bands="band", width=600, height=600)

## Top of Atmosphere (TOA) vs. Surface Reflectance & Atmospheric Correction
**TOA reflectance** refers to the reflectance values measured by a satellite sensor **before any correction for atmospheric effects**. It represents the **radiance reaching the sensor at the top of the atmosphere**, and includes contributions from:

* Direct solar reflectance from the surface,
* **Scattering by atmospheric molecules** (Rayleigh scattering),
* **Scattering by aerosols** and haze,
* **Reflection from clouds** and surrounding areas.

> **Sentinel-2 Level-1C** products are provided in TOA reflectance.

**Surface Reflectance** is the fraction of incoming solar radiation **reflected by the Earth's surface**, as it would appear **if the atmosphere were not present**. It represents the true reflectance of land, water, or vegetation and is more reliable for quantitative analysis.

> **Sentinel-2 Level-2A** products are atmospherically corrected to surface reflectance.

Sentinel-2 Level-2A data is produced using the Sen2Cor processor, which performs atmospheric correction to derive surface reflectance from top-of-atmosphere (TOA) reflectance. It was designed primarily for land surfaces — not water. We'll come back to this in a moment.

**Atmospheric correction** is the process of **removing atmospheric effects** from TOA reflectance to estimate **surface reflectance**. It compensates for:

* **Scattering** (blue light by molecules, haze),
* **Absorption** (by gases like water vapor, ozone, CO₂),
* **Aerosols** (dust, smoke, sea salt).

There are two main types:

* **Empirical methods**: e.g., **Dark Object Subtraction (DOS)**, based on scene content.
* **Physics-based methods**: e.g., **Sen2Cor**, **6S**, or **ACOLITE**, which use radiative transfer models and ancillary data (e.g., aerosol optical thickness).

Water bodies have unique optical properties that make atmospheric correction far more complex:

1. Low Reflectance (Dark Target Problem)
* Water reflects very little sunlight, especially in the visible and near-infrared.
* This makes it hard to distinguish water signal from atmospheric scattering, leading to unstable or noisy surface reflectance values.
* Overcorrected water pixels may even appear as negative reflectance, which is physically meaningless.

2. High Sensitivity to Atmospheric Effects
* Even small amounts of aerosols, thin clouds, or Rayleigh scattering can dominate the signal received from water.
* Sen2Cor is tuned for land reflectance properties, and may overcorrect these subtle signals in aquatic environments.

3. Incorrect Assumptions
* Sen2Cor assumes a Lambertian (diffuse) surface, which is true for land but not for water, which reflects sunlight specularly (like a mirror).
* It also uses land-based visibility and elevation models that don’t apply well to open water.

4. No Water-Specific Tuning
* Sen2Cor doesn’t use water-leaving radiance models or bidirectional reflectance functions (BRDF) specific to aquatic systems.
* It lacks aerosol correction schemes optimized for open water, unlike ocean color processors (e.g., ACOLITE or C2RCC).

#### What To Use Instead Over Water?
For more reliable atmospheric correction over water, use:

* ACOLITE – Optimized for coastal and inland waters.

* C2RCC (Case 2 Regional CoastColour) – Designed for complex waters with variable optical properties.

* Dark Object Subtraction (DOS) – A simple, empirical method suitable for ocean scenes with no clouds and minimal aerosols.

Today, we'll use DOS as it doesn't require us to download any external software. IT isn't perfect for this task, as we'll explain in next, but it does well enough to be suitable for this exercise.

### Dark Object Subtraction

**Dark Object Subtraction (DOS)** is a simple and widely used atmospheric correction method that estimates and removes the effects of atmospheric scattering in satellite imagery. It is particularly useful for correcting **Sentinel-2 Level-1C (Top-of-Atmosphere reflectance)** imagery in oceanic or coastal environments.

The core idea behind DOS is that certain "dark objects" in the scene—such as deep, clear water or dense vegetation—should theoretically have **near-zero reflectance** in some bands (especially the blue and shortwave bands). Any non-zero signal observed in these dark areas is attributed to **path radiance** (light scattered by the atmosphere).

#### How It Works

1. **Identify dark pixels**: Find the minimum (or low-percentile, e.g., 1%) reflectance values over dark surfaces, usually in the **blue band (Band 2)** or **coastal aerosol band (Band 1)**.
2. **Estimate path radiance**: Assume this value represents atmospheric scattering (haze, Rayleigh scattering).
3. **Subtract offset**: Subtract this value from all pixels in the scene for each band to estimate surface reflectance:

   ```
   ρ_surface = ρ_TOA - ρ_min
   ```
4. **Clip negatives**: Any resulting negative reflectance values are set to zero.

#### Application Over the Ocean

* **Clear deep water** serves as an ideal dark object.
* Works best in **clear-sky, deep-ocean** conditions where water reflectance is minimal.
* Helps reduce haze and Rayleigh scattering impacts in **shorter wavelengths** (e.g., blue, green).
* Less effective in **turbid, shallow, or coastal** waters where bottom reflectance or suspended particles are present.
* Simple, fast, and does not require external atmospheric data.
* Useful for preprocessing imagery in remote areas with limited ancillary data.

### Limitations

* Assumes perfect dark targets exist in the scene.
* Overcorrects in bright or turbid water conditions.
* Ignores adjacency effects and variable atmospheric thickness.
* Does not correct for absorption effects (e.g., water vapor, aerosols).


In [ ]:
def dark_object_subtraction(band, percentile=1, max_subtract=0.05):
    """
    Apply DOS safely to float32 reflectance images.
    Caps subtraction to prevent overcorrection.
    """
    # Sample spatially
    sample = band.isel(x=slice(None, None, 10), y=slice(None, None, 10))
    
    # Compute dark object reflectance
    dark_val = sample.quantile(percentile / 100.0, skipna=True).compute()
    dark_val = min(max(dark_val.item(), 0.0), max_subtract)
    print("Dark value:", dark_val)

    # Apply DOS (clip only min to avoid upward clipping)
    return (band - dark_val).clip(min=0)


def correct_dataset(ds, bands=["blue", "green", "red", "nir"]):
    corrected = {}
    for b in bands:
        corrected[b] = dark_object_subtraction(ds[b])
    return xr.Dataset(corrected, coords=ds.coords)


In [ ]:
# Sentinel-2 L1C data must be scaled from 0–10000 to 0–1 reflectance before applying dark object subtraction
ds_scaled = ds_masked / 10000.0
# Apply the correction on a single timestep (no .values used)
ds_corr = correct_dataset(ds_scaled.isel(time=0)) #.compute()

In [ ]:
# Check output ranges
vals = ds_masked["blue"].isel(time=0).values
vals_clean = vals[np.isfinite(vals)]

print("Unique:", np.unique(vals_clean))
print("Min:", vals_clean.min())
print("Max:", vals_clean.max())
print("Proportion of 1.0 values:", (vals_clean == 1.0).sum() / len(vals_clean))

In [ ]:
# Select and concatenate RGB bands
ds_rgb = ds_corr[["red", "green", "blue"]]
rgb = xr.concat([ds_rgb[b] for b in ["red", "green", "blue"]], dim="band")
rgb = rgb.assign_coords(band=["red", "green", "blue"])

# Sampled quantile scaling
scale = rgb.quantile(0.99, dim=("x", "y"), skipna=True).compute()
rgb_scaled = (rgb / scale).clip(0, 1)

# Plot
rgb_scaled.hvplot.rgb(x="x", y="y", bands="band", width=600, height=600)

### Compute Reef Health Indicators

The **Blue-Green Index (BGI)** is a spectral index used primarily in aquatic and coastal remote sensing to help discriminate benthic habitats (like coral reefs, seagrass, sand) and assess water properties. It leverages the difference and sum of the **blue** and **green** spectral bands.

#### Formula

$$
\text{BGI} = \frac{\text{Blue} - \text{Green}}{\text{Blue} + \text{Green}}
$$

* Calculated using surface reflectance values.


#### Interpretation of BGI values

| BGI Value Range        | Meaning / Typical Interpretation                                                                                  |
| ---------------------- | ----------------------------------------------------------------------------------------------------------------- |
| Positive (close to +1) | Blue reflectance > Green reflectance: Often clearer, deeper water or areas with more blue dominance               |
| Around 0               | Blue and green reflectance are about equal                                                                        |
| Negative (close to -1) | Green reflectance > Blue reflectance: Shallow water, vegetation (seagrass/algae), or sediments with greenish hues |

#### How BGI helps with coral reefs

* **Differentiates benthic substrates:** Coral, seagrass, algae, sand, and bare substrate often have distinct blue and green reflectance signatures, so BGI can help separate these classes.
* **Highlights shallow water features:** Coral reefs usually occur in shallow, clear water where blue and green light penetrates well, making BGI sensitive to benthic composition.
* **Tracks changes over time:** By analyzing BGI time series, you can detect changes in reef cover, algal blooms, sedimentation, or coral bleaching events affecting reflectance.


#### Limitations and considerations

* Usually masked to water pixels (e.g., using land/water masks or cloud masks).
* Values should be interpreted relative to local calibration or ground truth.
* **Water column effects:** Water depth, turbidity, and dissolved materials affect blue and green light differently, potentially confounding BGI values.
* **Atmospheric correction needed:** Accurate surface reflectance is essential for reliable BGI values, especially over water.
* **Not a direct health measure:** BGI reflects substrate and water color but not coral health metrics like bleaching directly. It is an indirect indicator.
* **Supplement with other indices and data:** Combine BGI with other indices (e.g., NDVI for algae, bathymetry data, or hyperspectral data) for better reef monitoring.

In [ ]:
# Visualize corrected BGI
def compute_bgi(ds):
    return (ds["blue"] - ds["green"]) / (ds["blue"] + ds["green"])

In [ ]:
bgi = compute_bgi(ds_corr)

In [ ]:
bgi.hvplot.image(x="x", y="y", cmap="viridis", width=600, height=400)

In [ ]:
# Check output ranges
vals = bgi.values
vals_clean = vals[np.isfinite(vals)]

print("Unique:", np.unique(vals_clean))
print("Min:", vals_clean.min())
print("Max:", vals_clean.max())
print("Proportion of 1.0 values:", (vals_clean == 1.0).sum() / len(vals_clean))

In [ ]:
# Save to GeoTIFF
bgi.rio.to_raster(f"bgi_{PROVINCE}_{DATERANGE_START}_{DATERANGE_END}.tif")


### Ground truth data integration and segmentation

We will use a reef map provided by Vanuatu Bureau of Statistics.

In [ ]:
benthic_gdf = gpd.read_file('Vanuatu reefs IMARS.zip')

In [ ]:
benthic_gdf.columns

In [ ]:
benthic_gdf.REEF.unique()

The reef map is a vector polygon dataset. We are rasterizing it to use as labels for the blue-green index pixels.

In [ ]:
width, height = bgi.x.size, bgi.y.size

benthic_gdf_rp = benthic_gdf.to_crs(epsg=bgi.rio.crs.to_epsg())

# Define the resolution and bounds based on BGI features
resolution = bgi.rio.resolution()
bounds_test = bgi.rio.bounds()

unique_classes = benthic_gdf['REEF'].unique()

raster_bounds = box(*bgi.rio.bounds())
benthic_gdf_select = benthic_gdf_rp[benthic_gdf_rp.intersects(raster_bounds)]

print(f"Before: {len(benthic_gdf_rp)} | After: {len(benthic_gdf_select)}")

# Rasterize the vector dataset to match the BGI image
rasterized_labels_benthic = make_geocube(
    vector_data=benthic_gdf_select,
    measurements=["REEF"], 
    like=bgi,  # Align with the features dataset
)

print("rasterized_labels_benthic: ", rasterized_labels_benthic)

Flatten the features (BGI) and newly rasterized labels for use with a random forest classifier.

In [ ]:
features = bgi.stack(flattened_pixel=("y", "x")).fillna(0)
labels = rasterized_labels_benthic.stack(flattened_pixel=("y", "x")).fillna(0).astype(int)

In [ ]:
labels = labels.to_array().squeeze()

In [ ]:
features.data.shape

In [ ]:
labels.data.shape

#### Data Splitting
Now that we have the arrays flattened, we can split the datasets into training and testing partitions. We will reserve 80 percent of the data for training, and 20 percent for testing.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42, shuffle=True
)

Ensure all labels are in each partition.

In [ ]:
np.unique(y_train), np.unique(y_test) 

In [ ]:
len(X_train), len(y_train), len(X_test), len(y_test)

Add the samples dimension. 

In [ ]:
X_train = X_train.data.reshape(-1, 1)
X_test = X_test.data.reshape(-1, 1)

Now we will set up a small [random forest classifider](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) with 10 trees. We use a [seed](https://towardsdatascience.com/why-do-we-set-a-random-state-in-machine-learning-models-bb2dc68d8431) (`random_state`) to ensure reproducibility. Calling the `.fit()` method on the classifier will initiate training.

In [ ]:
%%time
# Train a Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42) #n_estimators=10
clf.fit(X_train, y_train)

Once the classifier is finished training, we can use it to make predictions on our test dataset.

In [ ]:
# Test the classifier
y_pred = clf.predict(X_test)

It's important to know how well our classifier performs relative to the true labels (`y_test`). For this, we can calculate the [accuracy metric](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) to measure agreement between the true and predicted labels.

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

We can also produce a [classification report](https://scikit-learn.org/1.7/modules/generated/sklearn.metrics.classification_report.html)
to check the precision, recall and F1 scores for each class.

In [ ]:
print(classification_report(y_true=y_test, y_pred=y_pred))

We can also plot a confusion matrix to explore per-class performance.

In [ ]:
ConfusionMatrixDisplay.from_predictions(
    y_true=y_test, y_pred=y_pred, normalize="true", values_format=".2f"
)

Save the model to file so that it can be loaded and reused again without needing to repeat training.

In [ ]:
# save to file
joblib.dump(clf, f"rf_vanuatu_coral_{PROVINCE}_{DATERANGE_START}_{DATERANGE_END}.pkl")

If we want to generate predictions for the entire dataset in order to plot a map of predicted reefs for the entire area of interest, we can do this using the test province dataset.

In [ ]:
y_pred = clf.predict(features)

In [ ]:
predicted_map = y_pred.reshape((height, width))
predicted_map_xr = xr.DataArray(data=predicted_map, coords=rasterized_labels_benthic.coords)
print(np.unique(y_pred))

In [ ]:
predicted_map_xr.hvplot.image(height=600, rasterize=True, cmap="Set1")

Vectorize and save the predicted reef map to a geojson file.

In [ ]:
# Convert array to int32
compatible_array = predicted_map_xr.astype("int32")

# Rasterize to polygons
polygons = list(
    rasterio.features.shapes(
        compatible_array.values,
        transform=compatible_array.rio.transform()
    )
)

# Convert to GeoDataFrame and filter for value == 1
prediction_gdf = gpd.GeoDataFrame(
    [{"geometry": shape(geom), "value": value} for geom, value in polygons if value == 1],
    crs="EPSG:32759"
)

# print unique values (should be [1])
print(prediction_gdf.value.unique())

# Save to GeoJSON
prediction_gdf.to_file(
    f"./predicted_coral_{PROVINCE}_{DATERANGE_START}_{DATERANGE_END}.geojson",
    driver="GeoJSON"
)